In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import yaml

In [ ]:
%matplotlib inline 

In [ ]:
from model.mrcnn.config import Config
from model.mrcnn import utils
import model.mrcnn.model as modellib
from model.mrcnn import visualize
from model.mrcnn.model import log
from model.mrcnn import data_generator as datagen
from model.mrcnn import image_meta as meta
from model.mrcnn import dataset

In [142]:
from data.pascalvoc.pascal_frcnn import PascalVOCDataset
from data.pascalvoc.preprocessing.utils import load_ids_frcnn

In [ ]:
from config.config import cfg
from config import config_utils

In [ ]:
from pprint import pprint

In [ ]:
EXP_FOLDER = '/home/caleml/partial_experiments/exp_20200214_1749_frcnn_pascal_extended_frcnn_pascal_extended'

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """
    Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

# Config

In [ ]:
config_path = os.path.join(EXP_FOLDER, 'config.yaml')
with open(config_path, 'r') as f_in:
    loaded_conf = yaml.load(f_in)

config_utils.update_config(loaded_conf)
pprint(cfg)

# Dataset

In [ ]:
# Validation dataset
dataset_val = PascalVOCDataset()
dataset_val.load_pascal(cfg.DATASET.PATH, 8, 'val', extended=True, cfg=cfg)
dataset_val.prepare()

# model

In [ ]:
cfg.BATCH_SIZE = 1

In [ ]:
model = modellib.MaskRCNN(mode='inference', 
                          cfg=cfg,
                          exp_folder=EXP_FOLDER)

In [ ]:
checkpoint_path = model.find_last()

In [ ]:
print("Loading weights from ", checkpoint_path)

In [ ]:
model.load_weights(checkpoint_path, by_name=True)

# examples

## gt image

In [ ]:
image_id = random.choice(dataset_val.image_ids)
print(image_id)
original_image, image_meta, gt_class_id, gt_bbox = datagen.load_image_gt(dataset_val, cfg, image_id)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)

visualize.display_instances(original_image, gt_bbox, gt_class_id, dataset_val.class_names, figsize=(8, 8))

## detected image

In [ ]:
results = model.detect([original_image], verbose=1)

print(results)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['class_ids'], dataset_val.class_names, r['scores'], ax=get_ax())

# eval

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, 1000)
image_ids = dataset_val.image_ids

APs = []
total_class_AP = np.zeros(cfg.DATASET.NB_CLASSES) -1
count = 0
class_data = load_ids_frcnn(extended=True)

for image_id in image_ids:
    count += 1
    if count % 100 == 0:
        print("done %s val images, temp AP %s" % (count, np.mean(APs)))
    
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox = datagen.load_image_gt(dataset_val, cfg, image_id)
    
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    
    # Compute AP
    AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, r["rois"], r["class_ids"], r["scores"])
    APs.append(AP)
    # print('AP %s' % AP)
    
    cls_aps = utils.compute_class_ap(gt_bbox, gt_class_id, r["rois"], r["class_ids"], r["scores"], nb_classes=cfg.DATASET.NB_CLASSES)
    # print(cls_aps)
    total_class_AP = np.vstack((total_class_AP, cls_aps))

    
print("mAP: ", np.mean(APs))
print('per class mAP')
per_class_map = [np.mean(total_class_AP[:,i][np.where(total_class_AP[:,i] != -1)]) for i in range(cfg.DATASET.NB_CLASSES)]
print(per_class_map)

for cname, cdata in class_data.items():
    print('class %s, class mAP %s' % (cname, per_class_map[cdata['id']]))

done 100 val images, temp AP 0.7085003173059095
done 200 val images, temp AP 0.6344985909432249
done 300 val images, temp AP 0.6313360424310432
done 400 val images, temp AP 0.6309665858140988
done 500 val images, temp AP 0.6304201007534038
done 600 val images, temp AP 0.6297862873374979
done 700 val images, temp AP 0.627427806672739
done 800 val images, temp AP 0.6359094919361779
done 900 val images, temp AP 0.6308818603152906
done 1000 val images, temp AP 0.629674044876222
done 1100 val images, temp AP 0.6302892968634715
done 1200 val images, temp AP 0.6292223053749249
done 1300 val images, temp AP 0.630606233033204
done 1400 val images, temp AP 0.6281133793137482
done 1500 val images, temp AP 0.6298896911684226
done 1600 val images, temp AP 0.6334290059527428
done 1700 val images, temp AP 0.6374230157027646


# debug

In [69]:
print(len(APs))

1


In [42]:
np.random.random()

0.20102427579585358

In [54]:
a = np.array([-1, 0, -1, 3, 2])

In [55]:
a

array([-1,  0, -1,  3,  2])

In [56]:
np.cumsum(a > -1)

array([0, 1, 1, 2, 3])

In [58]:
np.arange(4) + 1

array([1, 2, 3, 4])

In [59]:
(0.25 * 0.33)  + (0.25*0.5) + (0.25*0.6)

0.35750000000000004

In [60]:
(0.25*0.5) + (0.5*0.6)

0.425

In [74]:
gt_boxes = np.array(['bla', 'bli', 'blu'])
gt_classes = np.array([15, 15, 7])

fil = np.where(gt_classes == 15)
filtered_boxes = gt_boxes[fil]

In [75]:
fil

(array([0, 1]),)

In [76]:
filtered_boxes

array(['bla', 'bli'], dtype='<U3')

In [79]:
a = np.zeros(3) -1
print(a)

[-1. -1. -1.]


In [86]:
len(a)

3

In [87]:
a.size

3

In [90]:
np.version.version

'1.17.2'

In [110]:
b = np.zeros(3) -1
print(b)

[-1. -1. -1.]


In [113]:
a[0] = 0 
a[2] = 0.33
b[1] = 0.5
b[2] = 0.66

In [114]:
print(a)
print(b)

[ 0.   -1.    0.33]
[-1.    0.5   0.66]


In [116]:
total = np.vstack((a, b))

In [117]:
c = np.array([0, -1, 0.5])
total = np.vstack((total, c))

In [118]:
print(total)

[[ 0.   -1.    0.33]
 [-1.    0.5   0.66]
 [ 0.   -1.    0.5 ]]


In [120]:
total.shape

(3, 3)

In [135]:
i = 2
np.mean(total[:,i][np.where(total[:,i] != -1)])

0.49666666666666665

In [129]:
total[:,1]

array([-1. ,  0.5, -1. ])

In [132]:
np.where(total[:,0] != -1)

(array([0, 2]),)